In [2]:
import tensorflow as tf
import numpy as np

from pathlib import Path
import os

2023-07-03 16:36:27.041486: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 16:36:27.074616: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 16:36:27.075358: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 16:36:27.633651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
path_to_save = Path("./trained_model.h5")
model = tf.keras.models.load_model(path_to_save)

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [13]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7oju9oty/assets


INFO:tensorflow:Assets written to: /tmp/tmp7oju9oty/assets
2023-07-03 19:33:28.435333: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-03 19:33:28.435358: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-03 19:33:28.435541: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp7oju9oty
2023-07-03 19:33:28.437655: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-07-03 19:33:28.437672: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp7oju9oty
2023-07-03 19:33:28.445143: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-03 19:33:28.518607: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp7oju9oty
2023-07-03 19:33:28.536160: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

In [10]:
tflite_model_dir = Path("./tflite_model_dir")
tflite_model_dir.mkdir(exist_ok=True, parents = True)

In [14]:
tflite_model_file = tflite_model_dir/"vgg16_model.tflite"
tflite_model_file.write_bytes(tflite_model)

59068320

In [16]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_model_dir/"vgg16_quant_model.tfliter"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpzxvxfs_c/assets


INFO:tensorflow:Assets written to: /tmp/tmpzxvxfs_c/assets
2023-07-03 19:38:44.565473: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-03 19:38:44.565493: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-03 19:38:44.565656: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpzxvxfs_c
2023-07-03 19:38:44.567765: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-07-03 19:38:44.567778: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpzxvxfs_c
2023-07-03 19:38:44.575209: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-03 19:38:44.652297: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpzxvxfs_c
2023-07-03 19:38:44.669535: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

14837480

In [17]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [19]:
import matplotlib.pyplot as plt
test_img = plt.imread("./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")

In [20]:
test_img.shape

(534, 800, 3)

In [23]:
interpreter.get_input_details()[0]["shape"]

array([  1, 224, 224,   3], dtype=int32)

In [26]:
resize_img = tf.image.resize(test_img, (224,224))

In [27]:
resize_img.shape

TensorShape([224, 224, 3])

In [28]:
input_data = tf.expand_dims(resize_img, axis=0)

In [30]:
input_data.shape

TensorShape([1, 224, 224, 3])

In [33]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [34]:
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)

In [35]:
pred 

array([[1.000000e+00, 1.156959e-09]], dtype=float32)

In [36]:
label_map = {0:"ants", 1:"bees"}

In [39]:
argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'ants'

In [40]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]



interpreter_quant.set_tensor(input_index, input_data)
interpreter_quant.invoke()
pred = interpreter_quant.get_tensor(output_index)


argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'ants'